In [ ]:
import re
import math
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import linkage, fcluster

In [ ]:
class HScipyHybridSemanticChunker2:
    def __init__(self, embeddings, similarity_threshold, cluster_threshold, window_size, transfer_sentence_count, debug=False):
        """
        :param embeddings: embed_query metoduna sahip bir embedding modeli (örn: OpenAIEmbeddings)
        :param similarity_threshold: Sliding window içindeki segmentlerin benzerlik eşiği (bu parametre artık referans olarak kalıyor; gerçek karar dinamik threshold ile veriliyor)
        :param cluster_threshold: Hiyerarşik kümeleme için benzerlik eşiği
        :param window_size: Sliding window'da kaç segmentin bir arada değerlendirileceği (aynı zamanda pencere boyutu)
        :param delimiters: Kural tabanlı bölme için kullanılacak regex paterni
        :param transfer_sentence_count: Lookahead yöntemiyle sınırdaki kaç cümlenin transferine bakılacağı
        :param debug: Debug çıktıları için True yapıldığında ek bilgiler yazdırılır.
        """
        self.embeddings = embeddings
        self.similarity_threshold = similarity_threshold
        self.cluster_threshold = cluster_threshold
        self.window_size = window_size
        self.transfer_sentence_count = transfer_sentence_count
        self.debug = debug

    def split_segment_by_word_boundary(self, segment, max_length):
        words = segment.split()
        parts = []
        current_part = ""
        for word in words:
            if len(current_part) + len(word) + 1 > max_length:
                parts.append(current_part.strip())
                current_part = word
            else:
                current_part = f"{current_part} {word}" if current_part else word
        if current_part.strip():
            parts.append(current_part.strip())
        return parts

    def preprocess_segments(self, segments):
        processed = []
        for seg in segments:
            seg = seg.strip()
            if len(seg) < 3:
                continue
            if len(seg) > 10000:
                processed.extend(self.split_segment_by_word_boundary(seg, 19000))
            else:
                processed.append(seg)
        return processed

    def split_text_into_sentences(self, text: str) -> list:
        """Metni cümlelere böler."""
        
        pattern = r'(?<=[a-zA-Z0-9])([.!])(?=\s*[A-Z])|(?<=\n)'
        temp_parts = re.split(pattern, text)
        
        # None değerleri boş stringe
        temp_parts = [part if part is not None else "" for part in temp_parts]
        
        # punc re-attach
        reattached_sentences = []
        i = 0
        while i < len(temp_parts):
            chunk = temp_parts[i]
            # Sonraki eleman sadece noktalama işareti ise bunu bu chunk'a ekle.
            if i + 1 < len(temp_parts) and re.match(r'^[.?!]$', temp_parts[i+1]):
                chunk += temp_parts[i+1]
                i += 1
            chunk = chunk.strip()
            if chunk:
                reattached_sentences.append(chunk)
            i += 1

        print(f"[DEBUG] split_text_into_sentences ile {len(reattached_sentences)} cümle bulundu: {reattached_sentences}")

        # Mevcut merging mantığı (1000 karakteri aşmayacak şekilde birleştirme)
        merged_sentences = []
        buffer = ""
        for sentence in reattached_sentences:
            if len(buffer) + len(sentence) < 1000:
                buffer = f"{buffer} {sentence}" if buffer else sentence
            else:
                if buffer:
                    merged_sentences.append(buffer)
                buffer = sentence

        if buffer:
            merged_sentences.append(buffer)

        return merged_sentences

In [ ]:
def rule_based_segmentation(self, text):
        # Önce metni cümlelere bölüp, sonra preprocess uygularız.
        segments = self.split_text_into_sentences(text)
        segments = self.preprocess_segments(segments)
        return segments

    def create_embeddings(self, texts: list) -> list:
        """
        Batch embedding metodu:
        Tüm metin parçalarını tek seferde embed_documents ile işleyerek performansı artırır.
        """
        return self.embeddings.embed_documents(texts)

    def calculate_dynamic_threshold_from_divergences(self, divergences):
        if not divergences:
            return 0.5
        mean_div = sum(divergences) / len(divergences)
        variance = sum((d - mean_div) ** 2 for d in divergences) / len(divergences)
        std_div = math.sqrt(variance)
        if std_div < 0.1:
            factor = 1.5
        elif std_div > 0.3:
            factor = 1.0
        else:
            factor = 1.25
        return mean_div + std_div * factor

In [ ]:
def semantic_merging(self, segments):
        """
        Bölünme mantığı:
          - Tüm segmentler üzerinde adım adım kayan pencere uygulanır (step = 1).
          - Her pencere için adjacent segmentler arası divergence değerleri hesaplanır.
          - Her pencere için yerel dinamik threshold belirlenir.
          - Eğer pencere içinde herhangi bir divergence, yerel threshold'u aşıyorsa,
            ilgili global index "split point" olarak kaydedilir.
          - Sonrasında split point'lere göre segmentler birleştirilerek chunk'lar oluşturulur.
        """
        n = len(segments)
        if n < self.window_size:
            if self.debug:
                print("[DEBUG] Segment sayısı pencere boyutundan küçük, tüm metin tek chunk olarak alınıyor.")
            return [" ".join(segments)]
        
        # Batch embedding: tüm segmentler için embedding'leri tek seferde alıyoruz.
        embeddings = self.create_embeddings(segments)
        split_points = set()
        
        # Kayan pencere: adım 1
        for window_start in range(0, n - self.window_size + 1):
            window_end = window_start + self.window_size
            window_embeddings = embeddings[window_start:window_end]
            window_divergences = []
            for i in range(self.window_size - 1):
                sim = cosine_similarity([window_embeddings[i]], [window_embeddings[i+1]])[0][0]
                divergence = 1 - sim
                window_divergences.append(divergence)
            local_threshold = self.calculate_dynamic_threshold_from_divergences(window_divergences)
            
            if self.debug:
                print(f"\n[DEBUG] Pencere {window_start}-{window_end} segmentleri: {segments[window_start:window_end]}")
                print(f"[DEBUG] Divergence değerleri: {window_divergences}")
                print(f"[DEBUG] Yerel dinamik threshold: {local_threshold:.4f}")
            
            # Pencere içindeki her adjacent divergence için kontrol
            for i, div in enumerate(window_divergences):
                if div > local_threshold:
                    global_index = window_start + i + 1
                    split_points.add(global_index)
                    if self.debug:
                        print(f"[DEBUG] Peak bulundu: Global index {global_index} (divergence: {div:.4f})")
        
        split_points = sorted(list(split_points))
        if self.debug:
            print(f"\n[DEBUG] Tespit edilen global split noktaları: {split_points}")
        
        # Split noktalarına göre chunk'ları oluştur
        chunks = []
        last_split = 0
        for point in split_points:
            chunk = " ".join(segments[last_split:point])
            if chunk:
                chunks.append(chunk)
                if self.debug:
                    print(f"[DEBUG] Chunk oluşturuldu: start={last_split}, end={point}, ilk 100 karakter: {chunk[:100]}...")
            last_split = point
        # Kalan parçayı ekle
        if last_split < n:
            chunk = " ".join(segments[last_split:])
            if chunk:
                chunks.append(chunk)
                if self.debug:
                    print(f"[DEBUG] Son Chunk: start={last_split}, end={n}, ilk 100 karakter: {chunk[:100]}...")
        return chunks

In [ ]:
def adjust_boundaries(self, chunks):
        """
        Batch yöntemiyle boundary adjustment:
        Her bir chunk çifti için, eğer sonraki chunk'ta (i+1) transfer_sentence_count'dan fazla cümle bulunuyorsa,
        sonraki chunk'ın ilk transfer_sentence_count cümlesi aday segment (candidate) olarak belirlenir,
        ve geri kalan kısmı (remainder) olarak alınır. Ardından, bu aday segmentin önceki chunk (i) ile olan benzerliği 
        (önceki chunk'in embedding'i ile karşılaştırılarak) ile aday segmentin remainder kısmı ile olan benzerliği 
        karşılaştırılır. Eğer aday segment, önceki chunk ile daha yüksek bir benzerlik (cosine similarity) gösteriyorsa,
        bu transfer_sentence_count cümle, sonraki chunk'tan kesilerek önceki chunk'e eklenir.
        """
        adjusted_chunks = chunks.copy()
        candidate_texts = []
        previous_texts = []
        remainder_texts = []
        indices = []
        
        # Her boundary için gerekli metinleri topla.
        for i in range(len(adjusted_chunks) - 1):
            next_sentences = self.split_text_into_sentences(adjusted_chunks[i+1])
            if not next_sentences or len(next_sentences) <= self.transfer_sentence_count:
                if self.debug:
                    print(f"[DEBUG] Chunk {i+1} sadece {len(next_sentences)} cümle içeriyor, boundary adjustment atlanıyor.")
                continue
            candidate_text = " ".join(next_sentences[:self.transfer_sentence_count])
            remainder = " ".join(next_sentences[self.transfer_sentence_count:])
            candidate_texts.append(candidate_text)
            previous_texts.append(adjusted_chunks[i])
            remainder_texts.append(remainder)
            indices.append(i)
        
        # Eğer batch için toplanan veri varsa, toplu embed işlemi yap.
        if candidate_texts:
            candidate_embeddings = self.create_embeddings(candidate_texts)
            previous_embeddings = self.create_embeddings(previous_texts)
            remainder_embeddings = self.create_embeddings(remainder_texts)
        
            # Her boundary için cosine similarity hesaplamalarını gerçekleştir.
            for idx, i in enumerate(indices):
                candidate_emb = candidate_embeddings[idx]
                prev_emb = previous_embeddings[idx]
                next_emb = remainder_embeddings[idx]
                sim_prev = cosine_similarity([prev_emb], [candidate_emb])[0][0]
                sim_next = cosine_similarity([next_emb], [candidate_emb])[0][0]
                
                if self.debug:
                    print(f"[DEBUG] Adjust Boundaries (Batch): Chunk {i} için aday text: '{candidate_texts[idx]}'")
                    print(f"[DEBUG] Önceki chunk ile similarity: {sim_prev:.4f}")
                    print(f"[DEBUG] Kalan kısım (remainder) ile similarity: {sim_next:.4f}")
                
                if sim_prev > sim_next:
                    # Transfer işlemini gerçekleştir.
                    next_sentences = self.split_text_into_sentences(adjusted_chunks[i+1])
                    candidate_text = " ".join(next_sentences[:self.transfer_sentence_count])
                    adjusted_chunks[i] = adjusted_chunks[i].strip() + " " + candidate_text
                    adjusted_chunks[i+1] = " ".join(next_sentences[self.transfer_sentence_count:])
                    if self.debug:
                        print(f"[DEBUG] Chunk {i+1}'den {self.transfer_sentence_count} cümle chunk {i}'e transfer edildi.")
        return adjusted_chunks

In [ ]:
def topic_based_refinement(self, chunks):
        if len(chunks) < 2:
            return chunks

        # Tüm chunk'ler için batch embedding işlemi
        vectors_start = time.time()
        vectors = self.create_embeddings(chunks)
        vectors = np.array(vectors)
        vectors_end = time.time()
        if self.debug:
            print(f"[DEBUG] Tüm chunk'lar için batch embedding süresi: {vectors_end - vectors_start:.4f} saniye.")

        # SciPy linkage çağrısı için zaman ölçümü
        linkage_start = time.time()
        Z = linkage(vectors, method='average', metric='cosine')
        linkage_end = time.time()
        if self.debug:
            print(f"[DEBUG] SciPy linkage toplam süresi: {linkage_end - linkage_start:.4f} saniye.")

        # SciPy fcluster çağrısı için zaman ölçümü
        fcluster_start = time.time()
        cluster_labels = fcluster(Z, t=self.cluster_threshold, criterion='distance')
        fcluster_end = time.time()
        if self.debug:
            print(f"[DEBUG] SciPy fcluster toplam süresi: {fcluster_end - fcluster_start:.4f} saniye.")

        if self.debug:
            print(f"[DEBUG] Linkage matrix: {Z}")
            print(f"[DEBUG] Cluster labels: {cluster_labels}")

        cluster_dict = {}
        chunk_indices = {}  # Her cluster için chunk indexlerini saklayalım
        chunk_contents = {}  # Her cluster için ilk 200 karakterlik içerikleri saklayalım

        for idx, (label, chunk) in enumerate(zip(cluster_labels, chunks)):
            cluster_dict.setdefault(label, []).append(chunk)
            chunk_indices.setdefault(label, []).append(idx)
            chunk_contents.setdefault(label, []).append(f"[Chunk {idx}]: {chunk[:200]}...")

        refined_chunks = []
        for label, cluster in cluster_dict.items():
            merged_chunk = " ".join(cluster)
            chunk_index_list = chunk_indices[label]
            chunk_content_list = chunk_contents[label]

            if len(cluster) > 1:
                if len(merged_chunk) > 10000:
                    # Birleşen chunk toplam uzunluğu 10,000 karakteri aşıyorsa, birleşmeden orijinal chunk'ları ekle
                    refined_chunks.extend(cluster)
                    if self.debug:
                        print(f"[DEBUG] Cluster {label} birleşmedi, çünkü toplam uzunluğu {len(merged_chunk)} karakter!")
                        print(f"[DEBUG] Bu cluster'daki chunk'lar ayrı kaldı: {chunk_index_list}")
                else:
                    refined_chunks.append(merged_chunk)
                    if self.debug:
                        print(f"[DEBUG] Cluster {label} birleşti! Birleşen chunk'lar: {chunk_index_list}")
                        print(f"[DEBUG] Chunk içerikleri:\n" + "\n".join(chunk_content_list))
            else:
                refined_chunks.append(merged_chunk)
                if self.debug:
                    print(f"[DEBUG] Cluster {label} tek başına kaldı: {chunk_index_list}")

        return refined_chunks

In [ ]:
def create_documents(self, texts):
        all_chunks = []
        for text in texts:
            # Adım 1: Preprocess + Dynamic Threshold
            step1_start = time.time()
            segments = self.rule_based_segmentation(text)
            initial_chunks = self.semantic_merging(segments)
            step1_total = time.time() - step1_start
            if self.debug:
                print(f"[DEBUG] Step 1 (Preprocess + Dynamic Threshold) toplam süresi: {step1_total:.4f} saniye.")
            
            # Adım 2: Adjust Boundaries
            step2_start = time.time()
            adjusted_chunks = self.adjust_boundaries(initial_chunks)
            step2_total = time.time() - step2_start
            if self.debug:
                print(f"[DEBUG] Step 2 (Adjust Boundaries) toplam süresi: {step2_total:.4f} saniye.")
            
            # Adım 3: Cluster (Topic-based Refinement)
            step3_start = time.time()
            refined_chunks = self.topic_based_refinement(adjusted_chunks)
            step3_total = time.time() - step3_start
            if self.debug:
                print(f"[DEBUG] Step 3 (Cluster) toplam süresi: {step3_total:.4f} saniye.")
            
            all_chunks.extend(refined_chunks)
        return all_chunks

In [ ]:
embedding_model_name = "text-embedding-3-large"
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY, model=embedding_model_name)

chunker = HScipyHybridSemanticChunker2(
    embeddings, 
    similarity_threshold=None, 
    cluster_threshold=0.08, 
    window_size=6,  
    transfer_sentence_count=2,
    debug=True
)